In [1]:
from models import * 
from Functions import * 
import h5py
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler

Using TensorFlow backend.


In [2]:
#Working on GPU (Preferably)
device =cuda()

GPU is available


In [ ]:
"""this can be used to download the files from Kaggle to google colab for example 
    Kaggle API key needs to be uploaded  """

from google.colab import files
upload=files.upload()
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle competitions download -c dreem-sleep-stages-2020 -p /content
#We need to decompress the files
!unzip \X_train.h5.zip

#  EEG Data Feature Extraction


In [4]:
f= h5py.File('X_train.h5', 'r')
L=list(f.keys())
y=pd.read_csv('y_train.csv')
y=y['sleep_stage'].values
df_eeg,n_measures_eeg,n_samples=Create_dataset(L[:7],f)
F_s=50 #Sampling Frequency
shape_eeg=(n_samples,n_measures_eeg,F_s*30)
X_train,y_train,X_val,y_val=Oversample(df_eeg,y,shape_eeg,split=.2)

In [5]:
batch_size=256
train_loader=create_loader(X_train,y_train,batch_size)
val_loader=create_loader(X_val,y_val,batch_size)
classifier_eeg = Feature_extractor(n_measures_eeg,F_s)
classifier_eeg.to(device)
optimizer = optim.Adam(classifier_eeg.parameters(),lr=0.1)#,weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
scheduler=lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.1)

In [6]:
#Training EEG data Feature EXtractor 
epochs=50
Losses_v=[]
Losses_t=[]
for epoch in range(epochs):
  print('epoch: ',epoch)
  Losses_t+=Train_model(classifier_eeg,criterion,optimizer,train_loader,scheduler,device,mode='train')
  Losses_v+=Train_model(classifier_eeg,criterion,optimizer,val_loader,scheduler,device,mode='val')
#Saving State
w_eeg=classifier_eeg.state_dict()

epoch:  0
train Loss 2.2396  Acc : 36.64%  f1 score 0.36
val Loss 1.5780  Acc : 41.86%  f1 score 0.37


# Accel+Pulsometer Data Feature Extraction


In [7]:
y_pul=pd.read_csv('y_train.csv')
y_pul=y_pul['sleep_stage'].values
df_pul,n_measures_pul,n_samples=Create_dataset(L[-4:],f)
F_s_pul=10 #Sampling Frequency
shape_pul=(n_samples,n_measures_pul,F_s_pul*30)
X_train_pul,y_train_pul,X_val_pul,y_val_pul=Oversample(df_pul,y_pul,shape_pul,split=.2)

In [8]:
batch_size=256
train_loader_pul=create_loader(X_train_pul,y_train_pul,batch_size)
val_loader_pul=create_loader(X_val_pul,y_val_pul,batch_size)
classifier_pul = Feature_extractor(n_measures_pul,F_s_pul)
classifier_pul.to(device)
optimizer = optim.Adam(classifier_pul.parameters(),lr=0.1)#,weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
scheduler=lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.1)

In [9]:
#Training Acceloremeter+Pulsometer Feature EXtractor 
epochs=30
Losses_v=[]
Losses_t=[]
for epoch in range(epochs):
  print('epoch: ',epoch)
  Losses_t+=Train_model(classifier_pul,criterion,optimizer,train_loader_pul,scheduler,device,mode='train')
  Losses_v+=Train_model(classifier_pul,criterion,optimizer,val_loader_pul,scheduler,device,mode='val')
#Saving State
w_pul=classifier_pul.state_dict()

epoch:  0
train Loss 2.2937  Acc : 36.06%  f1 score 0.36
val Loss 1.3472  Acc : 35.74%  f1 score 0.34


# LSTM Sequential   training


In [10]:
#Creating Sequential Datasets Here
lookback=3
dataX,dataY=Create_Sequential_data(df_eeg,lookback=lookback,n_measures=n_measures_eeg,labels=y)
dataX_pul,dataY_pul=Create_Sequential_data(df_pul,lookback=lookback,n_measures=n_measures_pul,labels=y_pul)
data=np.concatenate((dataX.reshape(dataX.shape[0:2]+(-1,)),dataX_pul.reshape(dataX_pul.shape[0:2]+(-1,))),axis=2)
"""We can use this To free Up memory if needed"""
#del dataX,dataX_pul
"""------------------"""
dataX_train,dataX_val,dataY_train,dataY_val=train_test_split(data,dataY,test_size=.3,stratify=dataY)
Seq_train_loader=create_loader(dataX_train,dataY_train,batch_size=batch_size)
Seq_val_loader=create_loader(dataX_val,dataY_val,batch_size=batch_size)
"""Same We can use this To free Up memory if needed"""
#del dataX_train,dataX_val,data
"""------------------"""

'------------------'

In [11]:
seq=Seq_learn(classifier_eeg.n_features+classifier_pul.n_features,lookback=lookback,drop=0)
seq.to(device)
criterion = nn.CrossEntropyLoss()

### First we Freeze the feature Extractors wheights

In [12]:
classifier_eeg.load_state_dict(w_eeg)
classifier_pul.load_state_dict(w_pul)
for param in classifier_eeg.parameters():
    param.requires_grad = False
for param in classifier_pul.parameters():
    param.requires_grad = False
optimizer=optim.Adam([{'params':classifier_eeg.parameters(),'lr':0.001},
            {'params':classifier_pul.parameters(),'lr':0.001},
            {'params':seq.parameters()}],lr=0.1)#,weight_decay=1e-4)
scheduler=lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.1)

In [14]:
epochs=1
Losses_v=[]
Losses_t=[]
models=(classifier_eeg,classifier_pul,seq)
for epoch in range(epochs):
  print('epoch: ',epoch)
  Losses_t+=Train_seq_model(models,criterion,optimizer,Seq_train_loader,scheduler,shape_eeg,shape_pul,device,lookback,mode='train')
  Losses_v+=Train_seq_model(models,criterion,optimizer,Seq_val_loader,scheduler,shape_eeg,shape_pul,device,lookback,mode='val')

epoch:  0
train Loss 4.2428  Acc : 49.63%  f1 score 0.49
val Loss 1.1075  Acc : 60.25%  f1 score 0.60


### Fine Tuning the feature Extractors wheights

In [15]:
for param in classifier_eeg.parameters():
    param.requires_grad = True
for param in classifier_pul.parameters():
    param.requires_grad = True
optimizer=optim.Adam([{'params':classifier_eeg.parameters(),'lr':0.001},
            {'params':classifier_pul.parameters(),'lr':0.001},
            {'params':seq.parameters()}],lr=0.01)#,weight_decay=1e-4)
scheduler=lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.1)

In [16]:
epochs=30
Losses_v=[]
Losses_t=[]
models=(classifier_eeg,classifier_pul,seq)
for epoch in range(epochs):
  print('epoch: ',epoch)
  Losses_t+=Train_seq_model(models,criterion,optimizer,Seq_train_loader,scheduler,shape_eeg,shape_pul,device,lookback,mode='train')
  Losses_v+=Train_seq_model(models,criterion,optimizer,Seq_val_loader,scheduler,shape_eeg,shape_pul,device,lookback,mode='val')

epoch:  0


RuntimeError: CUDA out of memory. Tried to allocate 462.00 MiB (GPU 0; 4.00 GiB total capacity; 2.54 GiB already allocated; 31.90 MiB free; 363.84 MiB cached)

### Gerating Testing Files For Kaggle 

In [19]:
"""If files were downloaded without Unzipping Uncomment"""
#!unzip \X_test.h5.zip

'If files were downloaded without Unzipping Uncomment'

In [21]:
f= h5py.File('X_test.h5', 'r')  
Preds=Test_seq_model((classifier_eeg,classifier_pul,seq),batch_size,f ,L[:7],L[-4:],F_s,F_s_pul,lookback,w_eeg,w_pul,device )
Preds.to_csv('submission.csv',index=True,header=True)